In [1]:
import pandas as pd

In [68]:
def KnuthMorrisPratt(text, pattern):

    '''Yields all starting positions of copies of the pattern in the text.
Calling conventions are similar to string.find, but its arguments can be
lists or iterators, not just strings, it returns all matches, not just
the first one, and it does not need the whole text in memory at once.
Whenever it yields, it will have read the text exactly up to and including
the match that caused the yield.'''

    # allow indexing into pattern and protect against change during yield
    pattern = list(pattern)

    # build table of shift amounts
    shifts = [1] * (len(pattern) + 1)
    shift = 1
    for pos in range(len(pattern)):
        while shift <= pos and pattern[pos] != pattern[pos-shift]:
            shift += shifts[pos-shift]
        shifts[pos+1] = shift

    print(shifts)
    # do the actual search
    startPos = 0
    matchLen = 0
    for c in text:
        while matchLen == len(pattern) or \
              matchLen >= 0 and pattern[matchLen] != c:
            startPos += shifts[matchLen]
            matchLen -= shifts[matchLen]
        matchLen += 1
        if matchLen == len(pattern):
            yield startPos

def search_list(text, pattern):
    matchLen = len(pattern)
    for i in range(len(text)):
        if (text[i:i+matchLen] == pattern): return i
    
    return -1

In [91]:
vocab_dict = {}
vocab_list = []
review_lines = []

i = 0
with open('reviews_sample.txt') as f:    
    for line in f.read().splitlines():
        vocab_index = []
        for word in line.split(' '):
            if word not in vocab_dict:                
                vocab_dict[word] = i
                word_index = i
                vocab_list.append(word)
                i += 1
            else:
                word_index = vocab_dict[word]
            vocab_index.append(word_index)
        
        review_lines.append(vocab_index)

In [ ]:
from tqdm import tqdm

MIN_SUPPORT = len(review_lines) * 0.01

def get_support(all_reviews, pattern):
    support = 0
    for review in all_reviews:
        if search_list(review, pattern):
            support += 1
    
    return support

L = []
S = []
L1 = []
S1 = []
for vocab in tqdm(range(len(vocab_list))):
    s = get_support(review_lines, [vocab])    
    if(s >= MIN_SUPPORT):
        L1.append([vocab])
        S1.append(s)

L.append(L1)
S.append(S1)


  6%|▌         | 1231/22104 [02:14<38:44,  8.98it/s]

In [219]:
 def apriori_gen(all_lines, LK_1, min_support):
    LK = []
    S = []
    for l1 in LK_1:
        for l2 in LK_1:
            if l1[:-1] == l2[:-1] and l1[-1] < l2[-1]:
                l = l1 + [l2[-1]]
                support = get_support(all_lines, l)
                if support >= min_support:
                    LK.append(l)
                    S.append(support)
    
    return LK, S

In [220]:
LK_1 = L1
while len(LK_1) > 0:
    print("Working on %d-Itemsets" % len(LK_1[0]))
    LK_1, SK_1 = apriori_gen(review_lines, LK_1, MIN_SUPPORT)
    if (len(LK_1) > 0):
        L.append(LK_1)
        S.append(SK_1)

Working on 1-Itemsets
Working on 2-Itemsets
Working on 3-Itemsets


In [221]:
with open('patterns_1.txt', 'w') as f:
    for i in range(0, len(L[0])):
        f.write("%d:%s"%(S[0][i],cate_list[L[0][i][0]]))
        f.write("\n")

In [222]:
with open('patterns_2.txt', 'w') as f:
    for i in range(0, len(L)):
        for j in range(0, len(L[i])):
            f.write("%d:%s"%(S[i][j], cate_list[L[i][j][0]]))
            for k in range(1, i + 1):
                f.write(";%s"%cate_list[L[i][j][k]])
            f.write("\n")